In [1]:
import numpy as np
import pandas as pd
from numpy import random
from pyspark.sql.types import ShortType
from pyspark.sql.types import FloatType
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
spark=SparkSession.builder.getOrCreate()
data = sc.textFile("/root/LDA/train.txt")

In [13]:
V = data.flatMap(lambda x : x.split(" ")).distinct()

In [14]:
V.saveAsTextFile("/root/LDA/V")

In [2]:
V = sc.textFile("/root/LDA/V").collect()

In [135]:
df = pd.DataFrame(random.randint(0, 1,(26171,8)))

In [81]:
def myMap(x):
    result = []
    words = x.split(" ")
    start = 0
    for word in words[0:len(words)-1]:
        item = str(V.index(word)) + "-"
        start = start+1
        end = min(start+5,len(words))
        for index in range(start,end):
            result.append(item+str(V.index(words[index])))
    return result
B = data.flatMap(lambda x : myMap(x))

In [87]:
B = B.collect()

In [137]:
Topics = pd.DataFrame(random.randint(0, 1,(1,8)))

In [138]:
topics = []
for i in range(798107) :
    if i % 10000 == 0 :
        print(i)
    topic = random.randint(0, 8)
    words = B[i].split("-")
    df.ix[int(words[0]),topic] = df.ix[int(words[0]),topic] + 1
    df.ix[int(words[1]),topic] = df.ix[int(words[1]),topic] + 1
    Topics[topic] = Topics[topic] + 1
    topics.append(topic)

0


/usr/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000


In [149]:
for it in range(10):
    for i in range(798107) :
        if i % 10000 == 0 :
            print(i)
        words = B[i].split("-")
        oldTopic = topics[i]
        df.ix[int(words[0]),oldTopic] = df.ix[int(words[0]),oldTopic] - 1
        df.ix[int(words[1]),oldTopic] = df.ix[int(words[1]),oldTopic] - 1
        Topics[oldTopic] = Topics[oldTopic] - 1
                
        p = (Topics + 0.1)*(df.ix[int(words[0])]+0.01)*(df.ix[int(words[1])]+0.01)/(2*Topics+26171*0.01)/(2*Topics+26171*0.01+1)
                
        new_p = []
        for k in range(8) :
             new_p.append(sum(p.ix[0,0:k]))
        u = random.random() * new_p[7]
        newTopic = 0
        for newTopic in range(8) :
            if u < new_p[newTopic] :
                break
        topics[i] = newTopic
        df.ix[int(words[0]),newTopic] = df.ix[int(words[0]),newTopic] + 1
        df.ix[int(words[1]),newTopic] = df.ix[int(words[1]),newTopic] + 1
        Topics[newTopic] = Topics[newTopic] + 1

0


/usr/lib/python2.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
6

In [150]:
result = (df+0.01)/(df.sum()+26171*0.01)
result.insert(0, 'index', result.index)
for i in range(8) :
    ind = result.sort_values(by = [i],ascending = False)[0:10].index
    print("")
    print "topic"+str(i),
    for j in range(10) :
        print V[ind[j]],


topic0 sports news football democracy com politics-society world information school soccer 
topic1 computer health computers education-science edu information research intel news school 
topic2 music culture-arts-entertainment engine com business research digital information reviews internet 
topic3 health information research gov cancer com medical computers business education-science 
topic4 business trade information market theory education-science management yahoo theoretical international 
topic5 education-science research science edu web school graduate information home culture-arts-entertainment 
topic6 wikipedia political politics-society encyclopedia system wiki edu party education-science research 
topic7 com news sports movie culture-arts-entertainment culture games music amazon film
